In [1]:
import json
from pymatgen import Lattice, Structure, Molecule, Composition
from pymatgen.io.vasp import Poscar
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer
import numpy as np
from monty.json import MontyEncoder, MontyDecoder
from monty.serialization import loadfn, dumpfn
import pandas as pd
from itertools import chain
import os
import pprint
from pymatgen.analysis.elasticity.elastic import ElasticTensor

In [2]:
#Load data from folder called "data"

d_3d=loadfn('data/jdft_3d-5-23-2018.json',cls=MontyDecoder)
d=loadfn('data/jdft_2d-5-23-2018.json',cls=MontyDecoder)
#d=loadfn('data/jdft_2d.json',cls=MontyDecoder)
df_metals = pd.read_csv('data/metals_list.csv') #list of all metals on periodic table

In [13]:
#Miscellaneous Functions

#checks if a space group has inversion symmetry
def has_inversion(group):
    ranges = ((2, 2), (10, 15), (47, 74), (83, 88), (123, 142), (147, 148), (162, 167), (175,176), (191, 194), (200, 206), (221, 230))
    nums = set(chain(*(range(start, end+1) for start, end in ranges)))
    return (group in nums)

#enables clickable urls in the dataframe
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

#Writes a poscar of a structure given the index "num" 
def poscar_wr(num,d):
    x= d[num]['final_str']
    fileout = Poscar(x)
    filename = d[num]['final_str'].formula.replace(" ", "") + '.vasp'
    fileout.write_file(filename)
    print("Wrote " + filename)
    
#Takes as input the dataframe and a list of elements (e.g. ['Mn','P','O'])
def search_for_element(df, el_list):
        searchfor = el_list
        df1 = df[df['Formula'].str.contains('|'.join(searchfor))]
        return df1
    
#Counts the number of metals per cell in a structure
def count_metals(d_i):
    comp = Composition(d_i['final_str'].formula.replace(" ", ""))
    comp_dict = comp.get_el_amt_dict()
    num_metals = 0.0
    for i in comp_dict:
        for index, row in df_metals.iterrows():
            if i == df_metals['Symbol'].iloc[index]:
                num_metals += comp_dict[i]
    return num_metals

#Gets Elastic Tensor in array form from json object
def get_et(elast_str=''):
    if elast_str == 'na':
        return 'na'
    else:
        cij=np.empty((6, 6), dtype=float)
        elast=np.array(elast_str.split(','),dtype='float')
        count=0
        for ii in range(6):
            for jj in range(6):
                cij[ii][jj]=elast[count]
                count=count+1
        et=ElasticTensor.from_voigt(cij)
        return et

#Calculates Universal Anisotropy Index from elastic tensor
def get_anisotropy(x):
    if x == 'na':
        return np.NaN
    else:
        try:
            return x.universal_anisotropy
        except:
            return np.NaN

#Returns a maximum in a list of Anisotropy indices for a given mpid
def get_anisotropy_list(dat_3d, mpid):
    anisotropy_list = []
    for i in dat_3d:
        if i['mpid'] == mpid:
            anisotropy_list.append(get_anisotropy(get_et(i['elastic'])))
            #pprint.pprint(i)
        x = np.array(anisotropy_list, dtype=np.float64)    
    return np.nanmax(x)

#Returns a maximum in a list of bulk OP band gaps for a given mpid
def get_bulk_gap_list(dat_3d, mpid):
    bulk_gap_list = []
    for i in dat_3d:
        if i['mpid'] == mpid:
            if i['op_gap'] == 'na':
                bulk_gap_list.append(np.NaN)
            else:
                bulk_gap_list.append(i['op_gap'])
        x = np.array(bulk_gap_list, dtype=np.float64)    
    return np.nanmax(x)

In [14]:
#Building Dataset

keylist = ['Formula','OP_Gap', 'Bulk_OP_Gap', 'Bulk_Anisotropy', 'Atoms_per_cell','MBJ_Gap',
           'Mag_mom_per_cell', 'Mag_mom_per_metal', 'Mag_mom_per_area','Final_Energy_per_atom',
           'Exf_Energy_per_area','Space_Group','Has_Inversion','Jarvis_URL','MP_URL']
param_dict ={}
for i in keylist:
    param_dict[i] = {'ison':True, 'paramlist':[]}

#Omit columns from calculation here by setting "ison" to False. Default is True.
param_dict['MBJ_Gap']['ison'] = False
param_dict['MP_URL']['ison'] = True
param_dict['Mag_mom_per_cell']['ison'] = False
param_dict['Mag_mom_per_metal']['ison'] = False
param_dict['Mag_mom_per_area']['ison'] = True
param_dict['Bulk_Anisotropy']['ison'] = True
param_dict['Final_Energy_per_atom']['ison'] =False
param_dict['Atoms_per_cell']['ison'] =False
param_dict['Space_Group']['ison'] =False
param_dict['Bulk_OP_Gap']['ison'] =False    
    
for i in d:
    #useful parameters
    stoichiometry = i['final_str'].formula
    cell_vol = i['final_str'].volume
    area = cell_vol/i['final_str'].lattice.c
    num_atoms = i['final_str'].num_sites
    space_group = i['final_str'].get_space_group_info()[1]
    jarvis_url = str("https://www.ctcms.nist.gov/~knc6/jsmol/")+str(i['jid'])+str(".html")
    mp_url=str("https://materialsproject.org/materials/")+str(i['mpid'])+str('/#')
    magnetic_moment = i['magmom']['magmom_out']
    relaxed_energy = i['fin_en']
    exfoliation_energy = i['exfoliation_en']
    OptB88vdW_band_gap = i['op_gap']
    mBJ_band_gap = i['mbj_gap']
    if param_dict['Mag_mom_per_cell']['ison'] == True: 
        number_of_metals = count_metals(i)
        if number_of_metals != 0.0:
            magnetic_mom_metal = magnetic_moment/number_of_metals
        else:
            magnetic_mom_metal = 0.0
    else:
        magnetic_mom_metal = 'na'
    if param_dict['Bulk_Anisotropy']['ison'] == True:
        anisotropy = get_anisotropy_list(d_3d, i['mpid'])
    else:
        anisotropy = np.NaN
    if param_dict['Bulk_OP_Gap']['ison'] == True:
        bulk_gap = get_bulk_gap_list(d_3d, i['mpid'])
    else:
        bulk_gap = np.NaN
    
    #building lists
    param_dict['Formula']['paramlist'].append(stoichiometry)
    param_dict['Atoms_per_cell']['paramlist'].append(num_atoms)
    param_dict['OP_Gap']['paramlist'].append(OptB88vdW_band_gap)
    param_dict['MBJ_Gap']['paramlist'].append(mBJ_band_gap)
    param_dict['Mag_mom_per_cell']['paramlist'].append(magnetic_moment)
    param_dict['Mag_mom_per_area']['paramlist'].append(magnetic_moment/area) 
    param_dict['Mag_mom_per_metal']['paramlist'].append(magnetic_mom_metal)
    param_dict['Final_Energy_per_atom']['paramlist'].append(relaxed_energy/num_atoms)
    param_dict['Exf_Energy_per_area']['paramlist'].append(exfoliation_energy/area)
    param_dict['Space_Group']['paramlist'].append(space_group)
    param_dict['Has_Inversion']['paramlist'].append(has_inversion(space_group))
    param_dict['Jarvis_URL']['paramlist'].append(jarvis_url)
    param_dict['MP_URL']['paramlist'].append(mp_url)
    param_dict['Bulk_Anisotropy']['paramlist'].append(anisotropy)
    param_dict['Bulk_OP_Gap']['paramlist'].append(bulk_gap)

c:\users\kory\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
c:\users\kory\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: All-NaN slice encountered
c:\users\kory\appdata\local\conda\conda\envs\py3\lib\site-packages\ipykernel_launcher.py:82: RuntimeWarning: All-NaN slice encountered


In [36]:
#Generating dataframe

param_dict['MBJ_Gap']['ison'] = False
param_dict['MP_URL']['ison'] = True
param_dict['Mag_mom_per_cell']['ison'] = False
param_dict['Mag_mom_per_metal']['ison'] = False
param_dict['Mag_mom_per_area']['ison'] = True
param_dict['Bulk_Anisotropy']['ison'] = True
param_dict['Final_Energy_per_atom']['ison'] =False
param_dict['Atoms_per_cell']['ison'] =False
param_dict['Space_Group']['ison'] =True
param_dict['Bulk_OP_Gap']['ison'] =False    

headers =[]
list_of_lists = []
for i in param_dict:
     if param_dict[i]['ison'] == True:
        headers.append(i)
        list_of_lists.append(param_dict[i]['paramlist'])

df = pd.DataFrame(list_of_lists)
df = df.transpose()
df.columns = headers
df_unfiltered = df.copy()

df = df[~df['OP_Gap'].isin(['na'])] #remove items with no band gap
#df = df[df['OP_Gap']>1] #Only include items with band gap > 1eV
df = df[~df['Has_Inversion'].isin([True])] #remove items with inversion symmetry


#Sort by desired parameter
#df = df.sort_values('OP_Gap',ascending=True)
#df = df.sort_values('Exf_Energy_per_area',ascending=True)
#df = df.sort_values('Final_Energy_per_atom',ascending=True)
#df = df.reindex(df.Mag_mom_per_area.abs().sort_values(ascending=False).index)
df = df.reindex(df.Bulk_Anisotropy.abs().sort_values(ascending=False).index)  #sort by absolute value of Anisotropy
#df = df.sort_values('Mag_mom_per_metal',ascending=False)
#df = df.sort_values('Space_Group',ascending=True)


#create dataframe with clickable urls
if param_dict['MP_URL']['ison'] == True:
    dfurl = df.style.format({'Jarvis_URL': make_clickable, 'MP_URL': make_clickable})
    
print("Number of Materials:",df.shape[0])
dfurl

Number of Materials: 108


,Formula,OP_Gap,Bulk_Anisotropy,Mag_mom_per_area,Exf_Energy_per_area,Space_Group,Has_Inversion,Jarvis_URL,MP_URL
210,Mo2 Br6,0.2715,-796.557,0.10011,2.37876,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6109.html,https://materialsproject.org/materials/mp-23312/#
34,Bi1 O2,1.9231,-88.0988,0.084919,13.6974,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-14459.html,https://materialsproject.org/materials/mvc-15971/#
350,B1 N1,4.3585,73.2175,0,13.0703,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-688.html,https://materialsproject.org/materials/mp-984/#
406,B1 P1 S4,1.5645,63.354,6.24013e-07,3.29613,16,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6190.html,https://materialsproject.org/materials/mp-27724/#
336,B1 N1,4.4818,-62.9937,-1.83221e-08,10.7925,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-60594.html,https://materialsproject.org/materials/mp-7991/#
120,Au1 C1 N1,2.1703,57.5359,1.94879e-06,9.41328,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-20029.html,https://materialsproject.org/materials/mp-29196/#
233,Al1 P1 S4,2.5522,38.4982,-4.14754e-08,2.9029,16,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6265.html,https://materialsproject.org/materials/mp-27462/#
76,Cr1 P2 S7,0.0419,13.7166,0.0506673,1.45931,5,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-60578.html,https://materialsproject.org/materials/mp-768680/#
109,Sb2 N2,2.066,10.6803,0,6.19199,1,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-28311.html,https://materialsproject.org/materials/mvc-15384/#
104,Sn4 S4,1.5751,8.56235,2.62958e-06,3.76629,39,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-19586.html,https://materialsproject.org/materials/mp-8781/#


In [37]:
search_for_element(df_unfiltered, ['V'])
#poscar_wr(165,d)

,Formula,OP_Gap,Bulk_Anisotropy,Mag_mom_per_area,Exf_Energy_per_area,Space_Group,Has_Inversion,Jarvis_URL,MP_URL
32,V1 O2,0.1437,12.1201,0.137325,8.08194,164,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-1...,https://materialsproject.org/materials/mvc-115...
82,V1 Br2,0.6285,0.621134,0.239419,6.13991,164,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-1...,https://materialsproject.org/materials/mp-9717...
254,V2 Cl6,0.0212,NaN,0.127645,2.33961,162,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-28117/#
296,V1 Br2 O1,0.0455,NaN,0.0760165,5.14691,47,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-32450/#
314,V1 Ag1 P2 Se6,0.434,-3.35551,0.0568334,2.1235,5,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-6543/#
318,V1 Se2,0.0512,3.30411,0.0601471,8.61838,164,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-694/#
347,V1 S2,0.1274,3.90266,0.0389269,9.75806,164,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-5...,https://materialsproject.org/materials/mp-9561/#
374,V2 Cl2 O2,0.0107,-5.04263,0.319108,3.11449,59,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-8...,https://materialsproject.org/materials/mp-25118/#
446,V1 Cl2,0.7098,0.548819,0.263471,6.29714,164,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-8...,https://materialsproject.org/materials/mp-22877/#
456,V2 Br2 O2,0.011,NaN,0.308538,3.18902,59,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6...,https://materialsproject.org/materials/mp-32497/#


In [17]:
#Write dataframe to Excel Sheet
#writer = pd.ExcelWriter('magnetic_2D_1eV.xlsx')
#df.to_excel(writer,'Sheet1')
#writer.save()

In [34]:
dfurl

,Formula,OP_Gap,Bulk_Anisotropy,Mag_mom_per_area,Exf_Energy_per_area,Space_Group,Has_Inversion,Jarvis_URL,MP_URL
551,Si4 S4,0.1563,-2070.49,-1.33544e-06,0.00100537,53,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-27847.html,https://materialsproject.org/materials/mp-1023900/#
210,Mo2 Br6,0.2715,-796.557,0.10011,2.37876,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6109.html,https://materialsproject.org/materials/mp-23312/#
12,Co2 O4,1.3905,-281.305,0.135911,0.120678,12,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-31379.html,https://materialsproject.org/materials/mp-556750/#
302,C2,0.531,111.836,0,13.3787,191,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-667.html,https://materialsproject.org/materials/mp-48/#
34,Bi1 O2,1.9231,-88.0988,0.084919,13.6974,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-14459.html,https://materialsproject.org/materials/mvc-15971/#
350,B1 N1,4.3585,73.2175,0,13.0703,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-688.html,https://materialsproject.org/materials/mp-984/#
406,B1 P1 S4,1.5645,63.354,6.24013e-07,3.29613,16,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-6190.html,https://materialsproject.org/materials/mp-27724/#
336,B1 N1,4.4818,-62.9937,-1.83221e-08,10.7925,187,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-60594.html,https://materialsproject.org/materials/mp-7991/#
120,Au1 C1 N1,2.1703,57.5359,1.94879e-06,9.41328,25,False,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-20029.html,https://materialsproject.org/materials/mp-29196/#
360,Te4 Mo2,0.0018,-50.3975,-0.000172843,3.92704,11,True,https://www.ctcms.nist.gov/~knc6/jsmol/JVASP-676.html,https://materialsproject.org/materials/mp-MoTe2Td/#
